In [1]:
import numpy as np
import pandas as pd

In [2]:
from pathlib import Path
path_1 = Path(r'D:\jio institute\q2\ml\final_project\dump_excel\valid')

In [3]:
df = pd.read_csv(path_1.joinpath('beta1.csv'), header=0, index_col=0).reset_index()

In [4]:
df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,FiO2,...,Patient_ID,ICULOS,SepsisLabel,HR_diff,SPO2_diff,Temp_diff,SBP_diff,MAP_diff,DBP_diff,Resp_diff
0,90.0,100.0,36.6,105.0,75.0,52.0,17.0,-5.5,24.6,0.21,...,101097,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90.0,100.0,36.6,105.0,75.0,52.0,17.0,-5.5,24.6,0.21,...,101097,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90.0,100.0,36.6,105.0,75.0,52.0,17.0,-5.5,24.6,0.50,...,101097,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,90.0,100.0,36.6,105.0,75.0,52.0,17.0,-5.5,24.6,0.21,...,101097,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,77.0,100.0,36.6,118.0,83.0,58.0,18.0,-5.5,24.6,0.21,...,101097,5,0,-13.0,0.0,0.0,13.0,8.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761990,83.0,99.0,36.7,121.0,77.0,54.0,22.0,-2.5,29.1,0.90,...,113911,22,0,0.5,-1.0,0.6,-35.5,-14.5,-12.0,-2.0
761991,80.0,92.0,36.3,102.0,73.0,51.0,24.0,-2.5,29.1,0.50,...,113911,23,0,-17.0,-8.0,-0.4,-14.0,-11.0,-7.0,2.0
761992,95.0,97.0,36.7,128.5,83.0,58.5,25.0,-2.5,29.1,0.90,...,113911,24,0,12.0,-2.0,0.0,7.5,6.0,4.5,3.0
761993,104.0,99.0,36.7,127.0,85.0,59.0,24.0,-2.5,29.1,0.90,...,113911,25,0,24.0,7.0,0.4,25.0,12.0,8.0,0.0


# Partial sofa score

In [5]:
def partial_score(df):
    ps_score=[]
    for row in df["Platelets"]:
        if row>=150:
            ps_score.append(0)
        elif row <150 and row >=100 :
            ps_score.append(1)
        elif row <100 and row >=50:
            ps_score.append(2)
        elif row <50 and row >=20:
            ps_score.append(3)
        elif row <20 and row >=0:
            ps_score.append(4)
        else:
            ps_score.append(0)

    bil=[] 

    for row in df["Bilirubin_total"]:
        if row<1.2:
            bil.append(0)
        elif row <=1.9 and row >=1.2 :
            bil.append(1)
        elif row <=5.9 and row >=2:
            bil.append(2)
        elif row <=11.9 and row >=6:
            bil.append(3)
        elif row >=12:
            bil.append(4)
        else:
            bil.append(0)

    crea=[]  
    for row in df["Creatinine"]:
        if row<1.2:
            crea.append(0)
        elif row <=1.9 and row >=1.2 :
            crea.append(1)
        elif row <=3.4 and row >=2:
            crea.append(2)
        elif row <4.9 and row >=3.5:
            crea.append(3)
        elif row >=5:
            crea.append(4)
        else:
            crea.append(0)

    res=[]
    for row in df["MAP"]:
        if row>=70:
            res.append(0)
        elif row <70:
            res.append(1)
        else:
            res.append(0)
    overall=[ps_score,bil,crea,res]
    sofa = [sum(i) for i in zip(*overall)]
    return sofa

# Quick sofa score

In [6]:
def qsofa(df):
    sofa1=[]
    for row in df["Resp"]:
        if row<22:
            sofa1.append(0)
        elif row >=22 :
            sofa1.append(1)
        else:
            sofa1.append(0)
    sofa2=[]
    for row in df["SBP"]:
        if row >100:
            sofa2.append(0)
        elif row <=100 :
            sofa2.append(1)
        else:
            sofa2.append(0)
    max(sofa2)
    qsofa=[]
    qsofa = [sofa1[i] + sofa2[i] for i in range(len(sofa1))]
    return qsofa

# GI and shock index

In [7]:
df["GI"]=df["Age"]*df["Platelets"]
df["shock_index"]=df["HR"]/df["SBP"]
df['partial_sofa'] =partial_score(df)
df['quick_sofa'] = qsofa(df)

# Vital parameters with difference of two

In [8]:
df["HR_diff"].fillna(df["HR_diff"].median(),inplace=True)
df["SPO2_diff"].fillna(df["SPO2_diff"].median(),inplace=True)
df["Temp_diff"].fillna(df["Temp_diff"].median(),inplace=True)
df["SBP_diff"].fillna(df["SBP_diff"].median(),inplace=True)
df["MAP_diff"].fillna(df["MAP_diff"].median(),inplace=True)
df["DBP_diff"].fillna(df["DBP_diff"].median(),inplace=True)
df["Resp_diff"].fillna(df["Resp_diff"].median(),inplace=True)

In [9]:
df.isnull().sum()

HR                 0
O2Sat              0
Temp               0
SBP                0
MAP                0
DBP                0
Resp               0
BaseExcess         0
HCO3               0
FiO2               0
pH                 0
PaCO2              0
SaO2               0
BUN                0
Calcium            0
Chloride           0
Creatinine         0
Glucose            0
Lactate            0
Magnesium          0
Phosphate          0
Potassium          0
Bilirubin_total    0
Hct                0
Hgb                0
PTT                0
WBC                0
Platelets          0
Hour               0
Age                0
Gender             0
Patient_ID         0
ICULOS             0
SepsisLabel        0
HR_diff            0
SPO2_diff          0
Temp_diff          0
SBP_diff           0
MAP_diff           0
DBP_diff           0
Resp_diff          0
GI                 0
shock_index        0
partial_sofa       0
quick_sofa         0
dtype: int64

In [9]:
df1=pd.read_csv("data1.csv")

In [10]:
df1

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,FiO2,...,SPO2_diff,Temp_diff,SBP_diff,MAP_diff,DBP_diff,Resp_diff,GI,shock_index,partial_sofa,quick_sofa
0,91.5,100.0,37.72,126.00,80.0,56.0,17.0,0.0,23.0,0.4,...,0.0,0.00,0.00,0.0,0.0,0.0,13545.18,0.726190,0,0
1,91.5,100.0,37.72,126.00,80.0,56.0,17.0,0.0,22.0,0.4,...,0.0,0.00,0.00,0.0,0.0,0.0,13545.18,0.726190,0,0
2,93.5,100.0,37.72,124.75,77.0,60.0,16.5,0.0,22.0,0.4,...,0.0,0.00,-1.25,-3.0,4.0,-0.5,10398.32,0.749499,0,0
3,91.0,100.0,36.28,150.00,90.0,66.0,17.0,0.0,23.0,0.4,...,0.0,-1.44,24.00,10.0,10.0,0.0,13545.18,0.606667,0,0
4,89.0,100.0,37.72,124.50,74.0,52.0,16.0,0.0,22.0,0.4,...,0.0,0.00,-0.25,-3.0,-8.0,-0.5,12519.03,0.714859,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690203,93.5,99.0,36.11,80.00,63.0,50.5,9.5,2.0,30.0,0.4,...,-1.0,0.00,-0.50,-1.0,0.0,-1.0,12179.40,1.168750,4,1
690204,85.0,100.0,36.11,84.00,64.0,49.0,10.0,2.0,30.0,0.4,...,1.0,0.17,2.50,-1.0,-3.5,0.5,12179.40,1.011905,4,1
690205,86.0,99.5,36.11,84.00,64.0,50.0,11.0,2.0,30.0,0.4,...,0.5,0.00,4.00,1.0,-0.5,1.5,12179.40,1.023810,4,1
690206,87.0,100.0,36.11,88.00,67.0,52.0,10.5,2.0,30.0,0.4,...,0.0,0.00,4.00,3.0,3.0,0.5,12179.40,0.988636,4,1


In [11]:
df.rename(columns = {'SepsisLabel':'sepsis_label'}, inplace = True)

In [12]:
result = df.append(df1)

C:\Users\Mangaleswara\AppData\Local\Temp\ipykernel_10696\2917003059.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = df.append(df1)


In [13]:
result

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,FiO2,...,SPO2_diff,Temp_diff,SBP_diff,MAP_diff,DBP_diff,Resp_diff,GI,shock_index,partial_sofa,quick_sofa
0,90.0,100.0,36.60,105.0,75.0,52.0,17.0,-5.5,24.6,0.21,...,0.0,0.00,0.0,0.0,0.0,0.0,6111.0,0.857143,3,0
1,90.0,100.0,36.60,105.0,75.0,52.0,17.0,-5.5,24.6,0.21,...,0.0,0.00,0.0,0.0,0.0,0.0,6111.0,0.857143,3,0
2,90.0,100.0,36.60,105.0,75.0,52.0,17.0,-5.5,24.6,0.50,...,0.0,0.00,0.0,0.0,0.0,0.0,9702.0,0.857143,0,0
3,90.0,100.0,36.60,105.0,75.0,52.0,17.0,-5.5,24.6,0.21,...,0.0,0.00,0.0,0.0,0.0,0.0,5166.0,0.857143,3,0
4,77.0,100.0,36.60,118.0,83.0,58.0,18.0,-5.5,24.6,0.21,...,0.0,0.00,13.0,8.0,6.0,1.0,5166.0,0.652542,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690203,93.5,99.0,36.11,80.0,63.0,50.5,9.5,2.0,30.0,0.40,...,-1.0,0.00,-0.5,-1.0,0.0,-1.0,12179.4,1.168750,4,1
690204,85.0,100.0,36.11,84.0,64.0,49.0,10.0,2.0,30.0,0.40,...,1.0,0.17,2.5,-1.0,-3.5,0.5,12179.4,1.011905,4,1
690205,86.0,99.5,36.11,84.0,64.0,50.0,11.0,2.0,30.0,0.40,...,0.5,0.00,4.0,1.0,-0.5,1.5,12179.4,1.023810,4,1
690206,87.0,100.0,36.11,88.0,67.0,52.0,10.5,2.0,30.0,0.40,...,0.0,0.00,4.0,3.0,3.0,0.5,12179.4,0.988636,4,1


In [15]:
result.to_csv("total.csv")

In [14]:
result.isnull().sum()

HR                 0
O2Sat              0
Temp               0
SBP                0
MAP                0
DBP                0
Resp               0
BaseExcess         0
HCO3               0
FiO2               0
pH                 0
PaCO2              0
SaO2               0
BUN                0
Calcium            0
Chloride           0
Creatinine         0
Glucose            0
Lactate            0
Magnesium          0
Phosphate          0
Potassium          0
Bilirubin_total    0
Hct                0
Hgb                0
PTT                0
WBC                0
Platelets          0
Hour               0
Age                0
Gender             0
Patient_ID         0
ICULOS             0
sepsis_label       0
HR_diff            0
SPO2_diff          0
Temp_diff          0
SBP_diff           0
MAP_diff           0
DBP_diff           0
Resp_diff          0
GI                 0
shock_index        0
partial_sofa       0
quick_sofa         0
dtype: int64